In [12]:
#Importing the required libraries 
import os, random, argparse 
from PIL import Image 
import imghdr 
import numpy as np 

**FUNCTION TO GET AVERAGE RGBOLD:**

1.Find the number of pixels in the image
2.Get colors as [(cnt1, (r1, g1, b1)), ...]
3.Find the sum ( [(c1*r1, c1*g1, c1*g2),...])
4.Find average using (sum(ci*bi)/np, sum(ci*gi)/np, sum(ci*ri)/np) which gives us [(c1*b1, c2*b2, ..), (c1*g1, c1*g2,...)...]
5.Return the average value



In [13]:
def getAverageRGBOld(image): 
  """ 
  Given PIL Image, return average value of color as (r, g, b) 
  """
  # no. of pixels in image 
  npixels = image.size[0]*image.size[1] 
  # get colors as [(cnt1, (r1, g1, b1)), ...] 
  cols = image.getcolors(npixels) 
  # get [(c1*b1, c1*g1, c1*g2),...] 
  sumRGB = [(x[0]*x[1][0], x[0]*x[1][1], x[0]*x[1][2]) for x in cols] 
  # calculate (sum(ci*ri)/np, sum(ci*gi)/np, sum(ci*bi)/np) 
  # the zip gives us [(c1*r1, c2*r2, ..), (c1*g1, c1*g2,...)...] 
  avg = tuple([int(sum(x)/npixels) for x in zip(*sumBGR)]) 
  return avg 

**FUNCTION TO GET AVERAGE RGB**

1.Get the image as numpy array
2.Get the shape
3.Find out the average

In [14]:
def getAverageRGB(image): 
  """ 
  Given PIL Image, return average value of color as (r, g, b) 
  """
  # get image as numpy array 
  im = np.array(image) 
  # get shape 
  w,h,d = im.shape 
  # get average 
  return tuple(np.average(im.reshape(w*h, d), axis=0)) 


**FUNCTION TO SPLIT IMAGE**

1.Find width and height
2.Make an image list
3.Generate the list of dimensions
4.Append the cropped image
5.Return the image

In [15]:
def splitImage(image, size): 
  """ 
  Given Image and dims (rows, cols) returns an m*n list of Images 
  """
  W, H = image.size[0], image.size[1] 
  m, n = size 
  w, h = int(W/n), int(H/m) 
  # image list 
  imgs = [] 
  # generate list of dimensions 
  for j in range(m): 
    for i in range(n): 
      # append cropped image 
      imgs.append(image.crop((i*w, j*h, (i+1)*w, (j+1)*h))) 
  return imgs 



**FUNCTION TO GET IMAGES**

1.Explicit the load so that we don't run into resource crunch
2.Force loading image data from file
3.Whenever we get an exception we are going to skip the file and print Invalid image
4.If everything goes good it will return the images

In [16]:
def getImages(imageDir): 
  """ 
  given a directory of images, return a list of Images 
  """
  files = os.listdir(imageDir) 
  images = [] 
  for file in files: 
    filePath = os.path.abspath(os.path.join(imageDir, file)) 
    try: 
    # explicit load so we don't run into resource crunch 
      fp = open(filePath, "rb") 
      im = Image.open(fp) 
      images.append(im) 
      # force loading image data from file 
      im.load() 
      # close the file 
      fp.close() 
    except: 
      # skip 
      print("Invalid image: %s" % (filePath,)) 
  return images 



**GET IMAGE FILE NAMES**

1.Explicit the load so that we don't run into resource crunch
2.Force loading image data from file
3.Whenever we get an exception we are going to skip the file and print Invalid image
4.If everything goes good it will return the images

In [17]:
def getImageFilenames(imageDir): 
  """ 
  given a directory of images, return a list of Image file names 
  """
  files = os.listdir(imageDir) 
  filenames = [] 
  for file in files: 
    filePath = os.path.abspath(os.path.join(imageDir, file)) 
    try: 
      imgType = imghdr.what(filePath) 
      if imgType: 
        filenames.append(filePath) 
    except: 
      # skip 
      print("Invalid image: %s" % (filePath,)) 
  return filenames 


**FUNCTION TO GET BEST MATCH INDEX**

1.Find the closest RGB value to input(based on x/y/z distance)
2.Find the index
3.Return the best match index



In [18]:
def getBestMatchIndex(input_avg, avgs): 
  """ 
  return index of best Image match based on RGB value distance 
  """

  # input image average 
  avg = input_avg 
    
  # get the closest RGB value to input, based on x/y/z distance 
  index = 0
  min_index = 0
  min_dist = float("inf") 
  for val in avgs: 
    dist = ((val[0] - avg[0])*(val[0] - avg[0]) +
        (val[1] - avg[1])*(val[1] - avg[1]) +
        (val[2] - avg[2])*(val[2] - avg[2])) 
    if dist < min_dist: 
      min_dist = dist 
      min_index = index 
    index += 1

  return min_index 

**FUNCTION TO CREATE IMAGE GRID**

1.Going to do the sanity check
2.Get maximum height and width of images by assuming they are not equal
3.Create output image
4.Paste the images

In [23]:
def createImageGrid(images, dims): 
  """ 
  Given a list of images and a grid size (m, n), create 
  a grid of images. 
  """
  m, n = dims 

  # sanity check 
  assert m*n == len(images) 

  # get max height and width of images 
  # ie, not assuming they are all equal 
  width = max([img.size[0] for img in images]) 
  height = max([img.size[1] for img in images]) 

  # create output image 
  grid_img = Image.new('RGB', (n*width, m*height)) 
    
  # paste images 
  for index in range(len(images)): 
    row = int(index/n) 
    col = index - n*row 
    grid_img.paste(images[index], (col*width, row*height)) 
    
  return grid_img 




**FUNCTION TO CREATE PHOTOMOSAIC**

1.Going to split target image using splitimage function
2.For each target (splited image) we are going to pick one from input
3.Calculate the average of input images
4.Calculate the average of traget sub-image
5.Find match index
6.Draw mosaic to image 
7.Return mosaic image

In [24]:
def createPhotomosaic(target_image, input_images, grid_size, reuse_images=True): 
  """ 
  Creates photomosaic given target and input images. 
  """

  print('splitting input image...') 
  # split target image 
  target_images = splitImage(target_image, grid_size) 

  print('finding image matches...') 
  # for each target image, pick one from input 
  output_images = [] 
  # for user feedback 
  count = 0
  batch_size = int(len(target_images)/10) 

  # calculate input image averages 
  avgs = [] 
  for img in input_images: 
    avgs.append(getAverageBGR(img)) 

  for img in target_images: 
    # target sub-image average 
    avg = getAverageBGR(img) 
    # find match index 
    match_index = getBestMatchIndex(avg, avgs) 
    output_images.append(input_images[match_index]) 
    # user feedback 
    if count > 0 and batch_size > 10 and count % batch_size is 0: 
      print('processed %d of %d...' %(count, len(target_images))) 
    count += 1
    # remove selected image from input if flag set 
    if not reuse_images: 
      input_images.remove(match) 

  print('creating mosaic...') 
  # draw mosaic to image 
  mosaic_image = createImageGrid(output_images, grid_size) 

  # return mosaic 
  return mosaic_image 



<>:29: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:29: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-24-66a34c51a40b>:29: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if count > 0 and batch_size > 10 and count % batch_size is 0:


**MAIN FUNCTION**

1.We are going to take the inputs(i.e. target image and input images)
2.Check for validilty of input images
3.Define grid size
4.Resizing the inputs
5.Creating photomosaic
6.giving it as output

In [25]:
# Gather our code in a main() function 
def main(): 
  # Command line args are in sys.argv[1], sys.argv[2] .. 
  # sys.argv[0] is the script name itself and can be ignored 

  # parse arguments 
  #parser = argparse.ArgumentParser(description='Creates a photomosaic from input images') 
  # add arguments 
  #parser.add_argument('--target-image', dest='target_image', required=True) 
  #parser.add_argument('--input-folder', dest='input_folder', required=True) 
  #parser.add_argument('--grid-size', nargs=2, dest='grid_size', required=True) 
  #parser.add_argument('--output-file', dest='outfile', required=False) 

  #args = parser.parse_args() 

  ###### INPUTS ###### 

  # target image 
  target_image = Image.open("input_image.jpg") 

  # input images folder
  print('reading input folder...') 
  input_images = getImages("mosaic_targets") 

  # check if any valid input images found 
  if input_images == []: 
    print('No input images found in input mosaic targets folder') 
    exit() 

  # shuffle list - to get a more varied output? 
  random.shuffle(input_images) 

  # size of grid 
  grid_size = (target_image.size[0]//3,target_image.size[1]//3) 

  # output 
  output_filename = 'mosaic.png'
  if True: 
    output_filename = "result.png"
    
  # re-use any image in input 
  reuse_images = True

  # resize the input to fit original image size? 
  resize_input = True

  ##### END INPUTS ##### 

  print('starting photomosaic creation...') 
    
  # if images can't be reused, ensure m*n <= num_of_images 
  if not reuse_images: 
    if grid_size[0]*grid_size[1] > len(input_images): 
      print('grid size less than number of images') 
      exit() 
    
  # resizing input 
  if resize_input: 
    print('resizing images...') 
    # for given grid size, compute max dims w,h of tiles 
    dims = (int(target_image.size[0]/grid_size[1]),int(target_image.size[1]/grid_size[0])) 
    print("max tile dims: %s" % (dims,)) 
    # resize 
    for img in input_images: 
      img.thumbnail(dims) 

  # create photomosaic 
  mosaic_image = createPhotomosaic(target_image, input_images, grid_size, 
                  reuse_images) 

  # write out mosaic 
  mosaic_image.save(output_filename, 'PNG') 

  print("saved output to %s" % (output_filename,)) 
  print('done.') 

In [26]:
# Standard boilerplate to call the main() function to begin 
# the program. 
if __name__ == '__main__': 
  main() 


reading input folder...
starting photomosaic creation...
resizing images...
max tile dims: (1, 5)
splitting input image...
finding image matches...
processed 40944 of 409440...
processed 81888 of 409440...
processed 122832 of 409440...
processed 163776 of 409440...
processed 204720 of 409440...
processed 245664 of 409440...
processed 286608 of 409440...
processed 327552 of 409440...
processed 368496 of 409440...
creating mosaic...
saved output to result.png
done.
